## Setup

In [1]:
import numpy as np
import cv2 as cv
import glob
from pathlib import Path
import json
import gradio as gr
import os


# Download the images
def in_colab() -> bool:
    try:
        import google.colab
        return True
    except Exception:
        return False

REPO_URL = "https://github.com/Tiromachelan/camera-pose-estimator"

if in_colab():
  if not Path("camera-pose-estimator/images").exists():
    !git clone {REPO_URL}

## Calibration

In [2]:
class Calibration:
    @staticmethod
    def calibrateCamera(image_files):
        print(f"calibrating data in folder {image_files}")
        # termination criteria
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        pattern_size = (9, 6)
        objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)
        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d point in real world space
        imgpoints = [] # 2d points in image plane.

        images = glob.glob(image_files + '/*.jpeg')

        gray = None
        for fname in images:
            img = cv.imread(fname)
            gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            #print(f"{'processing image '}{fname}")

            # Find the chess board corners
            ret, corners = cv.findChessboardCorners(gray, pattern_size, None)

            # If found, add object points, image points (after refining them)
            if ret == True:
                print(f"{"Processed "}{fname}")
                objpoints.append(objp)

                corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
                imgpoints.append(corners2)
        return(cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None))

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def calibrate_and_display(folder_path):
    ret, mtx, dist, rvecs, tvecs = Calibration.calibrateCamera(folder_path)

    files = glob.glob(os.path.join(folder_path, "*.jpeg")) + glob.glob(os.path.join(folder_path, "*.JPEG"))

    results = {
        "return_value": ret,
        "camera_matrix": mtx,
        "distortion_coefficients": dist,
        "rotation_vectors": rvecs,
        "translation_vectors": tvecs,
        "image_paths": files
    }
    json_output = json.dumps(results, cls=NumpyEncoder, indent=4)

    with open("calibration.json", "w") as f:
        f.write(json_output)

    return json_output

calibrate_and_display("camera-pose-estimator/images")

calibrating data in folder camera-pose-estimator/images
Processed camera-pose-estimator/images/DSCN0496.jpeg
Processed camera-pose-estimator/images/DSCN0484.jpeg
Processed camera-pose-estimator/images/DSCN0486.jpeg
Processed camera-pose-estimator/images/DSCN0504.jpeg
Processed camera-pose-estimator/images/DSCN0501.jpeg
Processed camera-pose-estimator/images/DSCN0502.jpeg
Processed camera-pose-estimator/images/DSCN0497.jpeg
Processed camera-pose-estimator/images/DSCN0498.jpeg
Processed camera-pose-estimator/images/DSCN0488.jpeg
Processed camera-pose-estimator/images/DSCN0483.jpeg
Processed camera-pose-estimator/images/DSCN0487.jpeg
Processed camera-pose-estimator/images/DSCN0499.jpeg
Processed camera-pose-estimator/images/DSCN0495.jpeg
Processed camera-pose-estimator/images/DSCN0485.jpeg
Processed camera-pose-estimator/images/DSCN0490.jpeg
Processed camera-pose-estimator/images/DSCN0494.jpeg
Processed camera-pose-estimator/images/DSCN0489.jpeg
Processed camera-pose-estimator/images/DSCN

'{\n    "return_value": 0.6168792517672342,\n    "camera_matrix": [\n        [\n            3036.308285453977,\n            0.0,\n            2283.8394401526543\n        ],\n        [\n            0.0,\n            3037.1664461337955,\n            1645.166985135081\n        ],\n        [\n            0.0,\n            0.0,\n            1.0\n        ]\n    ],\n    "distortion_coefficients": [\n        [\n            0.030416991383825545,\n            -0.19095457258449586,\n            -0.007194285898023687,\n            -0.00042200107268990995,\n            0.5365693954294233\n        ]\n    ],\n    "rotation_vectors": [\n        [\n            [\n                -0.47447686858139637\n            ],\n            [\n                0.8932044770496111\n            ],\n            [\n                1.9840092780013256\n            ]\n        ],\n        [\n            [\n                -0.1480727438912805\n            ],\n            [\n                1.170968349313555\n            ],\n 